## __기본옵션__

In [ ]:
from google.colab import drive
drive.mount ('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os, glob
import numpy as np
import pandas as pd
from PIL import Image
import pickle
import cv2
import matplotlib.pyplot as plt
import warnings 
from time import time
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Input, Activation, Flatten
from tensorflow.keras.layers import BatchNormalization,Add,Dropout
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.models import Model, load_model
# from tensorflow.keras.layers import LeakyReLU, ReLU, Conv2D, MaxPooling2D, BatchNormalization, Conv2DTranspose, UpSampling2D, concatenate
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from keras.models import Sequential

Using TensorFlow backend.


In [ ]:
dir = "/content/drive/My Drive/KPMG/classifier"
os.chdir(dir)

In [ ]:
os.getcwd()

'/content/drive/My Drive/KPMG/classifier'

## __데이터 불러오기__

In [ ]:
# pickle파일 read
with open("/content/drive/My Drive/KPMG/classifier/pickle/batang.pkl","rb") as pk:
  batang_pk = pickle.load(pk)
with open("/content/drive/My Drive/KPMG/classifier/pickle/dotum.pkl","rb") as pk:
  dotum_pk = pickle.load(pk)
with open("/content/drive/My Drive/KPMG/classifier/pickle/gothic.pkl","rb") as pk:
  gothic_pk = pickle.load(pk)
with open("/content/drive/My Drive/KPMG/classifier/pickle/myeongjo.pkl","rb") as pk:
  myeongjo_pk = pickle.load(pk)


In [ ]:
# 전체 이미지 array 저장해놓은 것
# image_all = batang_pk+dotum_pk+gothic_pk+myeongjo_pk
batang_pk=np.array(batang_pk)
dotum_pk=np.array(dotum_pk)
gothic_pk=np.array(gothic_pk)
myeongjo_pk=np.array(myeongjo_pk)

In [ ]:
# seed설정
np.random.seed(0)

# 인덱스 만들어 주고 순서를 랜덤으로 바꾸어 인덱스를 출력해주는 함수
def make_index_permutation(font_pk):
  result=np.random.permutation(np.arange(len(font_pk)))
  return result


In [ ]:
batang_index=make_index_permutation(batang_pk)
dotum_index=make_index_permutation(dotum_pk)
gothic_index=make_index_permutation(gothic_pk)
myeongjo_index=make_index_permutation(myeongjo_pk)

In [ ]:
import math

## font_index를 slicing으로 split 해주고 리스트에 추가해주는 함수

def data_split_merge(split_cnt,font_index):
  result=[]
  for i in range(0,split_cnt):
    result.append(font_index[round((i*(len(font_index))/split_cnt)):round((i+1)*(len(font_index))/split_cnt)])
  return result


In [ ]:
split_cnt = 8

batang_index_list=data_split_merge(split_cnt,batang_index)
dotum_index_list=data_split_merge(split_cnt,dotum_index)
gothic_index_list=data_split_merge(split_cnt,gothic_index)
myeongjo_index_list=data_split_merge(split_cnt,myeongjo_index)


In [ ]:
# 원하는 갯수로 train data로 나눠주는 함수

def make_train_data(split_cnt,font_pk,font_index_list):
  result=[]
  for i in range(split_cnt):
    result.append(font_pk[font_index_list[i]])
  return result

In [ ]:
batang_train=make_train_data(split_cnt,batang_pk,batang_index_list)
dotum_train=make_train_data(split_cnt,dotum_pk,dotum_index_list)
gothic_train=make_train_data(split_cnt,gothic_pk,gothic_index_list)
myeongjo_train=make_train_data(split_cnt,myeongjo_pk,myeongjo_index_list)

total_train=[]
for i in range(split_cnt):
  total_train.append(batang_train[i].tolist()+dotum_train[i].tolist()+gothic_train[i].tolist()+myeongjo_train[i].tolist())

In [ ]:
total_df_list=[]

for i in range(split_cnt):
  batang_df= pd.DataFrame({'image':batang_train[i], 'category':0})
  dotum_df = pd.DataFrame({'image':dotum_train[i], 'category':1})
  gothic_df = pd.DataFrame({'image':gothic_train[i], 'category':2})
  myeongjo_df = pd.DataFrame({'image':myeongjo_train[i], 'category':3})
  total_df=pd.concat([batang_df,dotum_df,gothic_df,myeongjo_df])
  total_df.reset_index(drop=True, inplace=True)
  total_df_list.append(total_df)

In [ ]:
X_list=[]

def make_X_list(total_train_data):
  img=[]
  for i in range(len(total_train_data)):
    img.append(cv2.resize(total_train_data[i],(75,75)))
  return img


In [ ]:
for i in range(split_cnt):
  X_list.append(make_X_list(total_train[i]))

## __모델링 (InceptionResNetV2)__

### Train, Validation Split

In [ ]:
def make_train_XY(X,total_df_list,idx):
  return X_list[idx],np.array(total_df_list[idx]['category'])

#### 아래 train_XY안에 있는 idx를 0~split_cnt-1까지 바꿔주면서 넣어주기만 하면 됨

X,Y=make_train_XY(X_list,total_df_list,0)
X=np.array(X)

In [ ]:
X.shape

(13806, 75, 75, 3)

In [ ]:
Y.shape

(13806,)

In [ ]:
# Train, valid set split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, 
                                                    stratify=Y ,random_state=1234)

### InceptionResNetV2 모델

#### optimizer 선정

##### adam

In [ ]:
# inception_resnet_v2 shape는 최소 75x75
from keras.applications.inception_resnet_v2 import InceptionResNetV2, decode_predictions

incep_resnet = InceptionResNetV2(include_top=False, input_shape=(75,75,3))
incep_resnet.trainable=True
# incep_resnet.summary()

from keras.models import Sequential
from keras import layers

model = Sequential()
model.add(incep_resnet)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

219062272/219055592 [==============================] - 15s 0us/step


In [ ]:
# model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
# def Time(model, train_X,train_Y, batch_size, n_epoch):

#   training_start_time = time()
#   trained_model = model.fit(train_X, train_Y,
#                             batch_size=batch_size, epochs=n_epoch,
#                             validation_split = 0.2)
#   training_end_time = time()
#   print("Training time finished.\n%d epochs in %12.2f" % (n_epoch,
#                                                           training_end_time - training_start_time))
#   training_time = training_end_time-training_start_time
#   return trained_model, training_time

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
def Time(model, train_X,train_Y, batch_size, n_epoch):

  training_start_time = time()
  trained_model = model.fit(train_X, train_Y,
                            batch_size=batch_size, epochs=n_epoch,
                            validation_split = 0.2)
  training_end_time = time()
  print("Training time finished.\n%d epochs in %12.2f" % (n_epoch,
                                                          training_end_time - training_start_time))
  training_time = training_end_time-training_start_time
  return trained_model, training_time

fitted_model,adam_time = Time(model,X_train,y_train,batch_size=500,n_epoch=20)

score = model.evaluate(X_test, y_test)
adam_loss = score[0]
adam_acc = score[1]
print(score)
print("loss : ", score[0])
print("accuracy : ", score[1])

##### nadam

In [ ]:
# inceptionResNetV2 불러오기
incep_resnet = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(75,75,3))
incep_resnet.trainable=True

model = Sequential()
model.add(incep_resnet)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

fitted_model,nadam_time = Time(model,X_train,y_train,batch_size=500,n_epoch=20)

score = model.evaluate(X_test, y_test)
nadam_loss = score[0]
nadam_acc = score[1]
print(score)
print("loss : ", score[0])
print("accuracy : ", score[1])

Train on 8835 samples, validate on 2209 samples
Epoch 1/20
8835/8835 [==============================] - 147s 17ms/step - loss: 1.5582 - accuracy: 0.5327 - val_loss: 415.2588 - val_accuracy: 0.2304
Epoch 2/20
8835/8835 [==============================] - 49s 6ms/step - loss: 0.5652 - accuracy: 0.7458 - val_loss: 123.1191 - val_accuracy: 0.2354
Epoch 3/20
8835/8835 [==============================] - 50s 6ms/step - loss: 0.2076 - accuracy: 0.9247 - val_loss: 2.1843 - val_accuracy: 0.2626
Epoch 4/20
8835/8835 [==============================] - 48s 5ms/step - loss: 0.0933 - accuracy: 0.9725 - val_loss: 1.4367 - val_accuracy: 0.3327
Epoch 5/20
8835/8835 [==============================] - 49s 6ms/step - loss: 0.1355 - accuracy: 0.9630 - val_loss: 4.4053 - val_accuracy: 0.6474
Epoch 6/20
8835/8835 [==============================] - 49s 6ms/step - loss: 0.1715 - accuracy: 0.9525 - val_loss: 0.7897 - val_accuracy: 0.7651
Epoch 7/20
8835/8835 [==============================] - 49s 6ms/step - loss:

##### SGD

In [ ]:
# inceptionResNetV2 불러오기
incep_resnet = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(75,75,3))
incep_resnet.trainable=True

model = Sequential()
model.add(incep_resnet)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
fitted_model,sgd_time = Time(model,X_train,y_train,batch_size=500,n_epoch=20)

score = model.evaluate(X_test, y_test)
sgd_loss = score[0]
sgd_acc = score[1]
print(score)
print("loss : ", score[0])
print("accuracy : ", score[1])

Train on 8835 samples, validate on 2209 samples
Epoch 1/20
8835/8835 [==============================] - 105s 12ms/step - loss: 1.2394 - accuracy: 0.4100 - val_loss: 1.4619 - val_accuracy: 0.3110
Epoch 2/20
8835/8835 [==============================] - 45s 5ms/step - loss: 1.0111 - accuracy: 0.5682 - val_loss: 1.3210 - val_accuracy: 0.3825
Epoch 3/20
8835/8835 [==============================] - 45s 5ms/step - loss: 0.8525 - accuracy: 0.6601 - val_loss: 1.2416 - val_accuracy: 0.4414
Epoch 4/20
8835/8835 [==============================] - 46s 5ms/step - loss: 0.7282 - accuracy: 0.7224 - val_loss: 1.1660 - val_accuracy: 0.4962
Epoch 5/20
8835/8835 [==============================] - 45s 5ms/step - loss: 0.6308 - accuracy: 0.7605 - val_loss: 1.1106 - val_accuracy: 0.5124
Epoch 6/20
8835/8835 [==============================] - 46s 5ms/step - loss: 0.5539 - accuracy: 0.7888 - val_loss: 1.0335 - val_accuracy: 0.5618
Epoch 7/20
8835/8835 [==============================] - 45s 5ms/step - loss: 0.4

##### Adadelta

In [ ]:
# inceptionResNetV2 불러오기
incep_resnet = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(75,75,3))
incep_resnet.trainable=True

model = Sequential()
model.add(incep_resnet)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adadelta", metrics=["accuracy"])
fitted_model,adadelta_time = Time(model,X_train,y_train,batch_size=500,n_epoch=20)

score = model.evaluate(X_test, y_test)
adadelta_loss = score[0]
adadelta_acc = score[1]
print(score)
print("loss : ", score[0])
print("accuracy : ", score[1])

Train on 8835 samples, validate on 2209 samples
Epoch 1/20
8835/8835 [==============================] - 134s 15ms/step - loss: 1.0013 - accuracy: 0.5718 - val_loss: 1.7366 - val_accuracy: 0.3531
Epoch 2/20
8835/8835 [==============================] - 47s 5ms/step - loss: 0.4127 - accuracy: 0.8196 - val_loss: 1.1271 - val_accuracy: 0.4749
Epoch 3/20
8835/8835 [==============================] - 47s 5ms/step - loss: 0.1101 - accuracy: 0.9620 - val_loss: 0.8900 - val_accuracy: 0.5935
Epoch 4/20
8835/8835 [==============================] - 47s 5ms/step - loss: 0.0360 - accuracy: 0.9920 - val_loss: 0.7485 - val_accuracy: 0.6822
Epoch 5/20
8835/8835 [==============================] - 47s 5ms/step - loss: 0.0065 - accuracy: 0.9986 - val_loss: 0.8173 - val_accuracy: 0.7098
Epoch 6/20
8835/8835 [==============================] - 47s 5ms/step - loss: 0.0023 - accuracy: 0.9995 - val_loss: 0.6430 - val_accuracy: 0.7809
Epoch 7/20
8835/8835 [==============================] - 47s 5ms/step - loss: 0.0

##### Rmsprop

In [ ]:
# inceptionResNetV2 불러오기
incep_resnet = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(75,75,3))
incep_resnet.trainable=True

model = Sequential()
model.add(incep_resnet)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
fitted_model,rmsprop_time = Time(model,X_train,y_train,batch_size=500,n_epoch=20)

score = model.evaluate(X_test, y_test)
rmsprop_loss = score[0]
rmsprop_acc = score[1]
print(score)
print("loss : ", score[0])
print("accuracy : ", score[1])

Train on 8835 samples, validate on 2209 samples
Epoch 1/20
8835/8835 [==============================] - 137s 15ms/step - loss: 2.0256 - accuracy: 0.4232 - val_loss: 2246.2246 - val_accuracy: 0.3214
Epoch 2/20
8835/8835 [==============================] - 47s 5ms/step - loss: 0.6525 - accuracy: 0.6761 - val_loss: 24.1972 - val_accuracy: 0.3169
Epoch 3/20
8835/8835 [==============================] - 47s 5ms/step - loss: 0.3266 - accuracy: 0.8704 - val_loss: 1.9583 - val_accuracy: 0.5061
Epoch 4/20
8835/8835 [==============================] - 46s 5ms/step - loss: 0.1286 - accuracy: 0.9546 - val_loss: 1.2685 - val_accuracy: 0.5600
Epoch 5/20
8835/8835 [==============================] - 47s 5ms/step - loss: 0.1646 - accuracy: 0.9470 - val_loss: 0.8884 - val_accuracy: 0.6053
Epoch 6/20
8835/8835 [==============================] - 47s 5ms/step - loss: 0.1053 - accuracy: 0.9732 - val_loss: 1.9543 - val_accuracy: 0.5813
Epoch 7/20
8835/8835 [==============================] - 47s 5ms/step - loss:

##### adagrad

In [ ]:
# inceptionResNetV2 불러오기
incep_resnet = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(75,75,3))
incep_resnet.trainable=True

model = Sequential()
model.add(incep_resnet)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adagrad", metrics=["accuracy"])
fitted_model,adagrad_time = Time(model,X_train,y_train,batch_size=500,n_epoch=20)

score = model.evaluate(X_test, y_test)
adagrad_loss = score[0]
adagrad_acc = score[1]
print(score)
print("loss : ", score[0])
print("accuracy : ", score[1])

Train on 8835 samples, validate on 2209 samples
Epoch 1/20
8835/8835 [==============================] - 125s 14ms/step - loss: 4.9414 - accuracy: 0.2859 - val_loss: 8335383517.6967 - val_accuracy: 0.3214
Epoch 2/20
8835/8835 [==============================] - 44s 5ms/step - loss: 0.9656 - accuracy: 0.5057 - val_loss: 395359.8263 - val_accuracy: 0.2499
Epoch 3/20
8835/8835 [==============================] - 44s 5ms/step - loss: 0.6441 - accuracy: 0.6165 - val_loss: 575.6168 - val_accuracy: 0.2345
Epoch 4/20
8835/8835 [==============================] - 44s 5ms/step - loss: 0.5534 - accuracy: 0.6800 - val_loss: 1.3410 - val_accuracy: 0.5745
Epoch 5/20
8835/8835 [==============================] - 44s 5ms/step - loss: 0.4127 - accuracy: 0.7862 - val_loss: 1.1970 - val_accuracy: 0.4165
Epoch 6/20
8835/8835 [==============================] - 44s 5ms/step - loss: 0.2895 - accuracy: 0.8747 - val_loss: 0.8959 - val_accuracy: 0.5609
Epoch 7/20
8835/8835 [==============================] - 44s 5ms/

모델저장 에러   
https://mylifemystudy.tistory.com/69

#### Training

##### 1번째 학습

In [ ]:
# inceptionResNetV2 불러오기
incep_resnet = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(75,75,3))
incep_resnet.trainable=True

model = Sequential()
model.add(incep_resnet)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

def Time(model, train_X,train_Y, batch_size, n_epoch):

  training_start_time = time()
  trained_model = model.fit(train_X, train_Y,
                            batch_size=batch_size, epochs=n_epoch,
                            validation_split = 0.2)
  training_end_time = time()
  print("Training time finished.\n%d epochs in %12.2f" % (n_epoch,
                                                          training_end_time - training_start_time))
  return trained_model

fitted_model=Time(model,X_train,y_train,batch_size=500,n_epoch=20)

Train on 8835 samples, validate on 2209 samples
Epoch 1/20
8835/8835 [==============================] - 142s 16ms/step - loss: 1.3580 - accuracy: 0.4964 - val_loss: 3819.3513 - val_accuracy: 0.2128
Epoch 2/20
8835/8835 [==============================] - 47s 5ms/step - loss: 0.6010 - accuracy: 0.7503 - val_loss: 11.0744 - val_accuracy: 0.3214
Epoch 3/20
8835/8835 [==============================] - 47s 5ms/step - loss: 0.1986 - accuracy: 0.9293 - val_loss: 4.6392 - val_accuracy: 0.3214
Epoch 4/20
8835/8835 [==============================] - 47s 5ms/step - loss: 0.0909 - accuracy: 0.9746 - val_loss: 1.8733 - val_accuracy: 0.2707
Epoch 5/20
8835/8835 [==============================] - 47s 5ms/step - loss: 0.0231 - accuracy: 0.9933 - val_loss: 3.3081 - val_accuracy: 0.3608
Epoch 6/20
8835/8835 [==============================] - 47s 5ms/step - loss: 0.0419 - accuracy: 0.9879 - val_loss: 1.8455 - val_accuracy: 0.5034
Epoch 7/20
8835/8835 [==============================] - 47s 5ms/step - loss:

In [ ]:
# 모델저장
from keras.models import model_from_json
model_json = model.to_json()
with open("/content/drive/My Drive/KPMG/classifier/model/incep_model_1.json", "w") as json_file:
  json_file.write(model_json)

# 가중치 저장
model.save_weights("/content/drive/My Drive/KPMG/classifier/model/incep_weight_1.h5")

##### 2번째 학습

In [ ]:
from tensorflow.compat.v2.keras.models import model_from_json

# 모델, weight 로드
json_file = open("/content/drive/My Drive/KPMG/classifier/model/incep_model_1.json", "r")
loaded_model_json = json_file.read()
json_file.close()

# json파일로부터 model load
loaded_model = model_from_json(loaded_model_json)

# model에 weight 로드하기
weight = '/content/drive/My Drive/KPMG/classifier/model/incep_weight_1.h5'
loaded_model.load_weights(weight)

In [ ]:
X,Y=make_train_XY(X_list,total_df_list,1)
X=np.array(X)

# Train, valid set split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, 
                                                    stratify=Y ,random_state=1234)

loaded_model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
fitted_model=Time(loaded_model,X_train,y_train,batch_size=500,n_epoch=20)

Epoch 1/20
18/18 [==============================] - 39s 2s/step - loss: 0.0305 - accuracy: 0.9912 - val_loss: 0.1155 - val_accuracy: 0.9633
Epoch 2/20
18/18 [==============================] - 34s 2s/step - loss: 0.0361 - accuracy: 0.9937 - val_loss: 0.1752 - val_accuracy: 0.9529
Epoch 3/20
18/18 [==============================] - 34s 2s/step - loss: 0.0056 - accuracy: 0.9983 - val_loss: 0.1435 - val_accuracy: 0.9692
Epoch 4/20
18/18 [==============================] - 35s 2s/step - loss: 0.0420 - accuracy: 0.9902 - val_loss: 0.2144 - val_accuracy: 0.9470
Epoch 5/20
18/18 [==============================] - 34s 2s/step - loss: 0.0061 - accuracy: 0.9975 - val_loss: 1.5034 - val_accuracy: 0.8004
Epoch 6/20
18/18 [==============================] - 34s 2s/step - loss: 0.1162 - accuracy: 0.9737 - val_loss: 0.1471 - val_accuracy: 0.9565
Epoch 7/20
18/18 [==============================] - 35s 2s/step - loss: 0.0092 - accuracy: 0.9980 - val_loss: 0.1087 - val_accuracy: 0.9679
Epoch 8/20
18/18 [==

KeyboardInterrupt: ignored

In [ ]:
# 모델저장
from keras.models import model_from_json
model_json = loaded_model.to_json()
with open("/content/drive/My Drive/KPMG/classifier/model/incep_model_2.json", "w") as json_file:
  json_file.write(model_json)

# 가중치 저장
loaded_model.save_weights("/content/drive/My Drive/KPMG/classifier/model/incep_weight_2.h5")


##### 3번째 학습

In [ ]:
from tensorflow.compat.v2.keras.models import model_from_json

# 모델, weight 로드
json_file = open("/content/drive/My Drive/KPMG/classifier/model/incep_model_2.json", "r")
loaded_model_json = json_file.read()
json_file.close()

# json파일로부터 model load
loaded_model = model_from_json(loaded_model_json)

# model에 weight 로드하기
weight = '/content/drive/My Drive/KPMG/classifier/model/incep_weight_2.h5'
loaded_model.load_weights(weight)

In [ ]:
X,Y=make_train_XY(X_list,total_df_list,2)
X=np.array(X)

# Train, valid set split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, 
                                                    stratify=Y ,random_state=1234)

loaded_model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
fitted_model=Time(loaded_model,X_train,y_train,batch_size=500,n_epoch=20)

In [ ]:
# 모델저장
from keras.models import model_from_json
model_json = loaded_model.to_json()
with open("/content/drive/My Drive/KPMG/classifier/model/incep_model_3.json", "w") as json_file:
  json_file.write(model_json)

# 가중치 저장
loaded_model.save_weights("/content/drive/My Drive/KPMG/classifier/model/incep_weight_3.h5")


##### 4번째 학습

In [ ]:
from tensorflow.compat.v2.keras.models import model_from_json

# 모델, weight 로드
json_file = open("/content/drive/My Drive/KPMG/classifier/model/incep_model_3.json", "r")
loaded_model_json = json_file.read()
json_file.close()

# json파일로부터 model load
loaded_model = model_from_json(loaded_model_json)

# model에 weight 로드하기
weight = '/content/drive/My Drive/KPMG/classifier/model/incep_weight_3.h5'
loaded_model.load_weights(weight)

In [ ]:
X,Y=make_train_XY(X_list,total_df_list,3)
X=np.array(X)

# Train, valid set split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, 
                                                    stratify=Y ,random_state=1234)

loaded_model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
fitted_model=Time(loaded_model,X_train,y_train,batch_size=500,n_epoch=20)

In [ ]:
# 모델저장
from keras.models import model_from_json
model_json = loaded_model.to_json()
with open("/content/drive/My Drive/KPMG/classifier/model/incep_model_4.json", "w") as json_file:
  json_file.write(model_json)

# 가중치 저장
loaded_model.save_weights("/content/drive/My Drive/KPMG/classifier/model/incep_weight_4.h5")


##### 5번째 학습

In [ ]:
from tensorflow.compat.v2.keras.models import model_from_json

# 모델, weight 로드
json_file = open("/content/drive/My Drive/KPMG/classifier/model/incep_model_4.json", "r")
loaded_model_json = json_file.read()
json_file.close()

# json파일로부터 model load
loaded_model = model_from_json(loaded_model_json)

# model에 weight 로드하기
weight = '/content/drive/My Drive/KPMG/classifier/model/incep_weight_4.h5'
loaded_model.load_weights(weight)

In [ ]:
X,Y=make_train_XY(X_list,total_df_list,4)
X=np.array(X)

# Train, valid set split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, 
                                                    stratify=Y ,random_state=1234)

loaded_model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
fitted_model=Time(loaded_model,X_train,y_train,batch_size=500,n_epoch=20)

In [ ]:
# 모델저장
from keras.models import model_from_json
model_json = loaded_model.to_json()
with open("/content/drive/My Drive/KPMG/classifier/model/incep_model_5.json", "w") as json_file:
  json_file.write(model_json)

# 가중치 저장
loaded_model.save_weights("/content/drive/My Drive/KPMG/classifier/model/incep_weight_5.h5")


##### 6번째 학습

In [ ]:
from tensorflow.compat.v2.keras.models import model_from_json

# 모델, weight 로드
json_file = open("/content/drive/My Drive/KPMG/classifier/model/incep_model_5.json", "r")
loaded_model_json = json_file.read()
json_file.close()

# json파일로부터 model load
loaded_model = model_from_json(loaded_model_json)

# model에 weight 로드하기
weight = '/content/drive/My Drive/KPMG/classifier/model/incep_weight_5.h5'
loaded_model.load_weights(weight)

In [ ]:
X,Y=make_train_XY(X_list,total_df_list,5)
X=np.array(X)

# Train, valid set split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, 
                                                    stratify=Y ,random_state=1234)

loaded_model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
fitted_model=Time(loaded_model,X_train,y_train,batch_size=500,n_epoch=20)

In [ ]:
# 모델저장
from keras.models import model_from_json
model_json = loaded_model.to_json()
with open("/content/drive/My Drive/KPMG/classifier/model/incep_model_6.json", "w") as json_file:
  json_file.write(model_json)

# 가중치 저장
loaded_model.save_weights("/content/drive/My Drive/KPMG/classifier/model/incep_weight_6.h5")


##### 7번째 학습

In [ ]:
from tensorflow.compat.v2.keras.models import model_from_json

# 모델, weight 로드
json_file = open("/content/drive/My Drive/KPMG/classifier/model/incep_model_6.json", "r")
loaded_model_json = json_file.read()
json_file.close()

# json파일로부터 model load
loaded_model = model_from_json(loaded_model_json)

# model에 weight 로드하기
weight = '/content/drive/My Drive/KPMG/classifier/model/incep_weight_6.h5'
loaded_model.load_weights(weight)

In [ ]:
X,Y=make_train_XY(X_list,total_df_list,6)
X=np.array(X)

# Train, valid set split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, 
                                                    stratify=Y ,random_state=1234)

loaded_model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
fitted_model=Time(loaded_model,X_train,y_train,batch_size=500,n_epoch=20)

In [ ]:
# 모델저장
from keras.models import model_from_json
model_json = loaded_model.to_json()
with open("/content/drive/My Drive/KPMG/classifier/model/incep_model_7.json", "w") as json_file:
  json_file.write(model_json)

# 가중치 저장
loaded_model.save_weights("/content/drive/My Drive/KPMG/classifier/model/incep_weight_7.h5")


##### 8번째 학습

In [ ]:
from tensorflow.compat.v2.keras.models import model_from_json

# 모델, weight 로드
json_file = open("/content/drive/My Drive/KPMG/classifier/model/incep_model_7.json", "r")
loaded_model_json = json_file.read()
json_file.close()

# json파일로부터 model load
loaded_model = model_from_json(loaded_model_json)

# model에 weight 로드하기
weight = '/content/drive/My Drive/KPMG/classifier/model/incep_weight_7.h5'
loaded_model.load_weights(weight)

In [ ]:
X,Y=make_train_XY(X_list,total_df_list,7)
X=np.array(X)

# Train, valid set split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, 
                                                    stratify=Y ,random_state=1234)

loaded_model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
fitted_model=Time(loaded_model,X_train,y_train,batch_size=500,n_epoch=20)

In [ ]:
# 모델저장
from keras.models import model_from_json
model_json = loaded_model.to_json()
with open("/content/drive/My Drive/KPMG/classifier/model/incep_model_8.json", "w") as json_file:
  json_file.write(model_json)

# 가중치 저장
loaded_model.save_weights("/content/drive/My Drive/KPMG/classifier/model/incep_weight_8.h5")